In [ ]:
#we want to work with the unite database

In [ ]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline

# location of this week's data and all the results produced by this notebook 
# - this should be a path relative to your working directory
data_dir = 'Data'

In [ ]:
#run this in the terminal
#scp svenzi@euler.ethz.ch:/cluster/home/svenzi/unite_database/uniteDB/classifier.qza ~/FermentYOU/Data/classifier.qza

In [ ]:
#job to submit the classification to Euler
cat > classify_taxonomy.sh << 'EOF'
#!/bin/bash
#SBATCH --job-name=classify_taxonomy
#SBATCH --time=04:00:00
#SBATCH --mem-per-cpu=32G
#SBATCH --cpus-per-task=4
#SBATCH --output=classify_%j.out
#SBATCH --error=classify_%j.err

# Exit on error
set -e

# Activate QIIME2 environment
source ~/.bashrc
conda activate qiime2-moshpit-2025.7

# Set variables
CLASSIFIER=/cluster/home/svenzi/classifier.qza
READS=/cluster/home/svenzi/data_dir/ITS/filtered-dada1-rep-seqs-highschool.qza
OUTPUT_DIR=/cluster/home/svenzi/taxonomy_output

# Create output directory if it doesn't exist
mkdir -p $OUTPUT_DIR

# Run classification
qiime feature-classifier classify-sklearn \
    --i-classifier $CLASSIFIER \
    --i-reads $READS \
    --o-classification $OUTPUT_DIR/taxonomy.qza \
    --p-n-jobs 4

# Create visualization
qiime metadata tabulate \
    --m-input-file $OUTPUT_DIR/taxonomy.qza \
    --o-visualization $OUTPUT_DIR/taxonomy.qzv

echo "Job completed successfully!"
EOF

# Then submit
sbatch classify_taxonomy.sh

In [ ]:
#copy the taxonomy files to JupyterHub, run this command in the terminal
#scp svenzi@euler.ethz.ch:/cluster/home/svenzi/taxonomy_output/taxonomy.qza Data/
#scp svenzi@euler.ethz.ch:/cluster/home/svenzi/taxonomy_output/taxonomy.qzv Data/

In [ ]:
#check out the type of the downloaded files
! qiime tools peek $data_dir/taxonomy.qza